In [15]:
#system libraries
import os
import sys
import random
import time
#selenium libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#recaptcha libraries
import speech_recognition as sr
import ffmpy
import requests
import urllib
import pydub

In [3]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [4]:
curp = input('Enter the CURP: ')

In [20]:
try:
    #create chrome driver
    driver = webdriver.Chrome('chromedriver')
    wait = WebDriverWait(driver, 10)
    #go to website
    driver.get("https://www.gob.mx/curp/")
    # enter the curp in the input
    wait.until(EC.presence_of_element_located(("xpath",'//*[@id="curpinput"]'))).send_keys(curp)
    #switch to recaptcha frame
    wait.until(EC.frame_to_be_available_and_switch_to_it(("xpath", '//*[@id="ember313"]/div/div/iframe')))
    #click on checkbox to activate recaptcha
    wait.until(EC.element_to_be_clickable(("xpath",'//*[@id="recaptcha-anchor"]/div[1]'))).click()
    # if the recaptcha isn't finished on the first try
    time.sleep(1)
    validar_captcha = wait.until(EC.presence_of_element_located(
                                ("xpath",'//*[@id="recaptcha-anchor"]/div[1]')
                                )).get_attribute('style')
    if not validar_captcha == 'display: none;':
        #switch to recaptcha audio control frame
        driver.switch_to.default_content()
        wait.until(EC.frame_to_be_available_and_switch_to_it(("xpath",'/html/body/div[3]/div[4]/iframe')))
        #click on audio challenge
        wait.until(EC.element_to_be_clickable(("xpath",'//*[@id="recaptcha-audio-button"]'))).click()
        #switch to recaptcha audio challenge frame
        driver.switch_to.default_content()
        wait.until(EC.frame_to_be_available_and_switch_to_it(("xpath","/html/body/div[3]/div[4]/iframe")))
        while True:
            #get the mp3 audio file
            src = wait.until(EC.presence_of_element_located(
                            ("xpath",'//*[@id="audio-source"]'))).get_attribute("src")
            #download the mp3 audio file from the source
            urllib.request.urlretrieve(src, "sample.mp3")
            #convert mp3 to wav
            sound = pydub.AudioSegment.from_mp3("./sample.mp3")
            sound.export("sample.wav", format="wav")
            #translate audio to text with google voice recognition
            sample_audio = sr.AudioFile("./sample.wav")
            r = sr.Recognizer()
            with sample_audio as source:
                audio = r.record(source)
            key=r.recognize_google(audio)
            #key in results and submit
            wait.until(EC.presence_of_element_located(
                        ("xpath",'//*[@id="audio-response"]'))).send_keys(key.lower())
            wait.until(EC.presence_of_element_located(
                        ("xpath",'//*[@id="audio-response"]'))).send_keys(Keys.ENTER)
            #if not answered correctly try again
            error = wait.until(EC.presence_of_element_located(
                                ("xpath",'/html/body/div/div/div[1]'))).text
            if not error == 'Multiple correct solutions required - please solve more.':
                break
    #click on submit bottom
    driver.switch_to.default_content()
    wait.until(EC.element_to_be_clickable(("xpath",'//*[@id="searchButton"]'))).click()
    #curp don't exist
    time.sleep(1)
    wait = WebDriverWait(driver, 1)
    wait.until(EC.presence_of_element_located(("xpath",'//*[@id="modalMessage"]')))
    print('CURP no valido.')
except NoSuchElementException:
    print('error')
except TimeoutException:
    #extract info
    table = driver.find_elements("xpath",'//*[@id="ember336"]/section/div[1]/div/div[2]/form/div[2]/div[1]/div/div[2]/table/tr')
    resultados = []
    for index in range(len(table)):
        values = table[index].find_elements("tag name", "td")
        resultados.append(values[1].text)
    print(resultados)

['OICL980815HDFLNN01', 'LEONEL', 'OLIVARES', 'CONCHILLOS', 'HOMBRE', '15/08/1998', 'MEXICO', 'DISTRITO FEDERAL', 'ACTA DE NACIMIENTO']
